In [13]:
import requests
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import re

이전에 했던 내용들을 복습합니다.

In [14]:
auth = '***'

In [15]:
# 당일 접수 10건
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}'
url = url_tmpl.format(auth=auth)
r= requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo,'list')
result.head()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,005940,Y,NH투자증권,NH투자증권,20180618,20180618000130,,증권발행실적보고서
1,140290,N,청광종건,청광종건,20180618,20180618600232,넥,[기재정정]주주총회소집결의
2,066430,K,와이오엠,와이오엠,20180618,20180618900231,코,주식명의개서정지(주주명부폐쇄)
3,066430,K,와이오엠,와이오엠,20180618,20180618900230,코,주주총회소집결의
4,223220,N,구름게임즈앤컴퍼니,구름게임즈앤컴퍼니,20180618,20180618600212,넥,증권발행결과(자율공시)


In [16]:
#당일 접수 100건(max)
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=100'
url = url_tmpl.format(auth=auth)
r= requests.get(url)
r= json.loads(r.text)
result = json_normalize(r,'list')
result

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,005940,Y,NH투자증권,NH투자증권,20180618,20180618000130,,증권발행실적보고서
1,140290,N,청광종건,청광종건,20180618,20180618600232,넥,[기재정정]주주총회소집결의
2,066430,K,와이오엠,와이오엠,20180618,20180618900231,코,주식명의개서정지(주주명부폐쇄)
3,066430,K,와이오엠,와이오엠,20180618,20180618900230,코,주주총회소집결의
4,223220,N,구름게임즈앤컴퍼니,구름게임즈앤컴퍼니,20180618,20180618600212,넥,증권발행결과(자율공시)
5,00164876,E,케이비증권,케이비증권,20180618,20180618000126,,투자설명서(일괄신고)
6,206640,K,바디텍메드,배준광,20180618,20180618000124,,임원ㆍ주요주주특정증권등소유상황보고서
7,00908012,E,농협경제지주,농협경제지주,20180618,20180618000122,공,[기재정정]대규모기업집단현황공시[연1회공시및1/4분기용(대표회사)]
8,01222821,E,대신알루텍,태원공인회계사감사반(제62호),20180618,20180618000121,,[기재정정]감사보고서 (2017.12)
9,01113523,E,제이알제17호기업구조조정부동산투자회사,삼정회계법인,20180618,20180618000117,,감사보고서 (2018.03)


In [17]:
#회사의 전체 사업+반기+분기보고서 100건

url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003&page_set=100'
code = '005930'
url = url_tmpl.format(auth=auth,code=code)
r= requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo,'list')
result

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,005930,Y,삼성전자,삼성전자,20180515,20180515001699,,분기보고서 (2018.03)
1,005930,Y,삼성전자,삼성전자,20180402,20180402005019,연,사업보고서 (2017.12)
2,005930,Y,삼성전자,삼성전자,20171114,20171114001694,,분기보고서 (2017.09)
3,005930,Y,삼성전자,삼성전자,20170814,20170814001832,,반기보고서 (2017.06)
4,005930,Y,삼성전자,삼성전자,20170515,20170515003806,,분기보고서 (2017.03)
5,005930,Y,삼성전자,삼성전자,20170331,20170331004518,연,사업보고서 (2016.12)
6,005930,Y,삼성전자,삼성전자,20161114,20161114001491,,분기보고서 (2016.09)
7,005930,Y,삼성전자,삼성전자,20160816,20160816001779,,반기보고서 (2016.06)
8,005930,Y,삼성전자,삼성전자,20160516,20160516003174,,분기보고서 (2016.03)
9,005930,Y,삼성전자,삼성전자,20160330,20160330003536,연,사업보고서 (2015.12)


In [18]:
df =result[:10]

for ix,row in df.iterrows():
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(row['rcp_no'])
    print(ix,url)

0 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180515001699
1 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402005019
2 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171114001694
3 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170814001832
4 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170515003806
5 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331004518
6 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20161114001491
7 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160816001779
8 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160516003174
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330003536


In [19]:
def get_report_doc_urls(rcp_no):
    doc_urls = []
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)
    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches:
        params.append(m.replace("'", "").replace(" ", "").split(","))
    doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 
    for p in params:
        if rcp_no == p[0]:
            doc_urls.append(doc_url_tmpl % tuple(p))
    return doc_urls

In [20]:
doc_urls = get_report_doc_urls('20170515003806')
for doc in doc_urls:
    print(doc)

http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=1&offset=564&length=4104&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=2&offset=12691&length=409&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=3&offset=13104&length=333222&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=4&offset=13194&length=234991&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=5&offset=248189&length=19537&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=6&offset=267730&length=4064&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=7&offset=271798&length=58143&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=8&offset=329945&length=7183&dtd=dart3.xsd
http://dart.fss.or.kr/rep

url which is used in code = http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=1&offset=564&length=4104&dtd=dart3.xsd
    

this code is in network>Doc>preview : "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 

DART 오픈 API 02 - 페이지 분석

In [21]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import json
from io import BytesIO

#### 종목 마스터
- krx 파일 다운로드는 두 가지로 나뉜다.
    - Generate OPT request
    - OTP 코드 받아서 Download 파일 load

In [56]:
import requests
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import re

In [34]:
def get_krx_stock_master():
    # STEP 01: Generate OTP
    gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
    gen_otp_data = {
        'name':'fileDown',
        'filetype':'xls',
        'url':'MKD/04/0406/04060100/mkd04060100_01',
        'market_gubun':'ALL', # ''ALL':전체, STK': 코스피
    }

    r = requests.post(gen_otp_url, gen_otp_data)
    code = r.content

    # STEP 02: download
    down_url = 'http://file.krx.co.kr/download.jspx'
    down_data = {
        'code': code,
    }

    r = requests.post(down_url, down_data)
    
    
    df = pd.read_excel(BytesIO(r.content), dtype={'종목코드': str, '업종코드': str})
    df = df[['종목코드', '기업명', '업종코드', '업종', '대표전화', '주소']]
    return df

df_master = get_krx_stock_master()
print("총 %d 종목" % len(df_master))
df_master.head()

총 2197 종목


,종목코드,기업명,업종코드,업종,대표전화,주소
0,060310,3S,032902,특수 목적용 기계 제조업,02-896-9474,서울특별시 금천구 시흥대로71길 30-1
1,095570,AJ네트웍스,147603,산업용 기계 및 장비 임대업,02-6363-9999,"서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)"
2,068400,AJ렌터카,147601,운송장비 임대업,1544-1600,서울특별시 구로구 서부샛길 822
3,006840,AK홀딩스,116409,기타 금융업,02-6923-2921,서울특별시 구로구 구로중앙로 152(구로동)
4,054620,APS홀딩스,116409,기타 금융업,031-776-1800,경기도 화성시 동탄면 동탄산단9길 23-12 (동탄면)


인증키와 헤더

In [67]:
auth = '9babdb53604382c25e7f8c67c6548a853fdb957f'

In [68]:
# 종목의 사업+반기+분기보고서
code = '011150' # CJ씨푸드	
url_tmpl = "http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003&page_set=100"
url = url_tmpl.format(auth=auth, code=code)
r = requests.get(url)
jo = json.loads(r.text)
result = json_normalize(jo, 'list')

print('총', len(result), '건')
result.head()    

총 87 건


,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,011150,Y,CJ씨푸드,CJ씨푸드,20180515,20180515001871,,분기보고서 (2018.03)
1,011150,Y,CJ씨푸드,CJ씨푸드,20180402,20180402004301,,사업보고서 (2017.12)
2,011150,Y,CJ씨푸드,CJ씨푸드,20171114,20171114001635,,분기보고서 (2017.09)
3,011150,Y,CJ씨푸드,CJ씨푸드,20170814,20170814001294,,반기보고서 (2017.06)
4,011150,Y,CJ씨푸드,CJ씨푸드,20170515,20170515003886,,분기보고서 (2017.03)


In [70]:
for idx,row in result[:10].iterrows():
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(row['rcp_no'])
    print(ix,url)

9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180515001871
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402004301
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171114001635
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170814001294
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170515003886
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331004236
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20161114002083
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160816001205
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160516002153
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330004074


In [74]:
import requests
import re

def get_report_doc_urls(rcp_no):
    doc_urls = []
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)
    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches: 
        params.append(m.replace("'", "").replace(" ", "").split(","))
   
    doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 

    for p in params:
        if rcp_no == p[0]:
            doc_urls.append( doc_url_tmpl % tuple(p) )
        
    return doc_urls

In [75]:
doc_urls = get_report_doc_urls('20170515003806')
for doc in doc_urls:
    print(doc)

http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=1&offset=564&length=4104&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=2&offset=12691&length=409&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=3&offset=13104&length=333222&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=4&offset=13194&length=234991&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=5&offset=248189&length=19537&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=6&offset=267730&length=4064&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=7&offset=271798&length=58143&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20170515003806&dcmNo=5653406&eleId=8&offset=329945&length=7183&dtd=dart3.xsd
http://dart.fss.or.kr/rep